In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import os

### Data loading and Exploration

In [45]:
# loading data csv file
df = pd.read_csv('data/train_transaction.csv')

In [46]:
# checking data samples and feature/column count
df.shape

(590540, 394)

In [47]:
# checking sample count against each class
# there is apparently class imbalance problem which we will deal with later in this notebook
df['isFraud'].value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

In [48]:
# printing data head
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Data Wrangling & Preprocessing

In [49]:
# printing column/feature names
for col in df.columns:
    print(col)

TransactionID
isFraud
TransactionDT
TransactionAmt
ProductCD
card1
card2
card3
card4
card5
card6
addr1
addr2
dist1
dist2
P_emaildomain
R_emaildomain
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
D2
D3
D4
D5
D6
D7
D8
D9
D10
D11
D12
D13
D14
D15
M1
M2
M3
M4
M5
M6
M7
M8
M9
V1
V2
V3
V4
V5
V6
V7
V8
V9
V10
V11
V12
V13
V14
V15
V16
V17
V18
V19
V20
V21
V22
V23
V24
V25
V26
V27
V28
V29
V30
V31
V32
V33
V34
V35
V36
V37
V38
V39
V40
V41
V42
V43
V44
V45
V46
V47
V48
V49
V50
V51
V52
V53
V54
V55
V56
V57
V58
V59
V60
V61
V62
V63
V64
V65
V66
V67
V68
V69
V70
V71
V72
V73
V74
V75
V76
V77
V78
V79
V80
V81
V82
V83
V84
V85
V86
V87
V88
V89
V90
V91
V92
V93
V94
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V138
V139
V140
V141
V142
V143
V144
V145
V146
V147
V148
V149
V150
V151
V152
V153
V154
V155
V156
V157
V158
V159
V160
V161
V162
V163
V164
V165
V166
V167

In [50]:
# checking data types of all features/columns
df.dtypes

TransactionID       int64
isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
dist1             float64
dist2             float64
P_emaildomain      object
R_emaildomain      object
C1                float64
C2                float64
C3                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               float64
                   ...   
V310              float64
V311              float64
V312              float64
V313              float64
V314              float64
V315              float64
V316              float64
V317        

In [51]:
# checking if data has any missing values
df.isna().sum().sum() > 0

True

In [52]:
# percentage of missing data in each column
(df.isna().sum()/df.shape[0])*100

TransactionID      0.000000
isFraud            0.000000
TransactionDT      0.000000
TransactionAmt     0.000000
ProductCD          0.000000
card1              0.000000
card2              1.512683
card3              0.265012
card4              0.267044
card5              0.721204
card6              0.266028
addr1             11.126427
addr2             11.126427
dist1             59.652352
dist2             93.628374
P_emaildomain     15.994852
R_emaildomain     76.751617
C1                 0.000000
C2                 0.000000
C3                 0.000000
C4                 0.000000
C5                 0.000000
C6                 0.000000
C7                 0.000000
C8                 0.000000
C9                 0.000000
C10                0.000000
C11                0.000000
C12                0.000000
C13                0.000000
                    ...    
V310               0.002032
V311               0.002032
V312               0.002032
V313               0.214888
V314               0

In [53]:
# dropping columns with more than 50% missing records (rest will be imputed later)
missing_records = ((df.isna().sum()/df.shape[0])*100).loc[((df.isna().sum()/df.shape[0])*100)>50.0]
df.drop(columns=missing_records.index,inplace=True)

In [54]:
# new shape of dataframe
df.shape

(590540, 220)

In [55]:
# printing no. of distinct values for each categorical variable
for col in df.columns:
    if df[col].dtype == 'object':
        print(col,df[col].unique().shape[0])

ProductCD 5
card4 5
card6 5
P_emaildomain 60
M1 3
M2 3
M3 3
M4 4
M6 3


In [56]:
# imputing categorical variables using mode
for col in df.columns:
    if df[col].dtype == 'object':
        df.loc[df[col].isna(),col] = df[col].mode().values[0]

In [66]:
# checking data types of remaining columns with missing values
df.dtypes.loc[df.isna().sum().loc[lambda x:x>0].index].unique()

array([dtype('float64')], dtype=object)

In [87]:
# determining 'int' and 'float' columns [all are showing 'float' at the momemtn because of 'nan' values]
int_cols = []
float_cols = []
for col in df.isna().sum().loc[lambda x:x>0].index:
    if np.array_equal(df[col].dropna(), df[col].dropna().astype(int)):
        int_cols.append(col)
    else:
        float_cols.append(col)

In [176]:
# one-hot encoding categorical columns to make them usable for regression (to impute numeric columns)
cat_cols = []
for col in df.columns:
    if df[col].dtype == 'object':
        cat_cols.append(col)
# get encoded columns
one_hot = pd.get_dummies(df[cat_cols])
# drop original columns as they are not needed anymore
df.drop(columns=cat_cols,inplace=True)
# join the encoded columns to original df
df = df.join(one_hot)

In [177]:
# checking new data dimensions
df.shape[1]

294

In [ ]:
df = df_tmp.copy()

In [183]:
# getting indices of records with null values against each 'int' column
int_col_nan_indices = {}
for col in int_cols:
    int_col_nan_indices[col] = df.loc[df[col].isnull()].index
    
# getting indices of records with null values against each 'float' column
float_col_nan_indices = {}
for col in float_cols:
    float_col_nan_indices[col] = df.loc[df[col].isnull()].index

In [201]:
# temporarily imputing null values in 'int' and 'float' columns using randomly seen values
for col in int_col_nan_indices.keys():
    observed_values = df.loc[df[col].notnull(), col]
    df.loc[int_col_nan_indices[col],col] = np.random.choice(observed_values, int_col_nan_indices[col].shape[0]
                                                            ,replace=True)
for col in float_col_nan_indices.keys():
    observed_values = df.loc[df[col].notnull(), col]
    df.loc[float_col_nan_indices[col],col] = np.random.choice(observed_values, float_col_nan_indices[col].shape[0]
                                                            ,replace=True)

In [212]:
# now using regression to impute missing values in numeric columns (ints and floats)
from sklearn import linear_model

for col in int_cols:
    X = df.loc[~df.index.isin(int_col_nan_indices[col])].drop(columns=col)
    y = df.loc[~df.index.isin(int_col_nan_indices[col]),col]
    model = linear_model.LinearRegression()
    model.fit(X, y)
    df.loc[df.index.isin(int_col_nan_indices[col]),col] = model.predict(df.loc[df.index.isin(int_col_nan_indices[col])].drop(columns=col))

In [213]:
for col in float_cols:
    X = df.loc[~df.index.isin(float_col_nan_indices[col])].drop(columns=col)
    y = df.loc[~df.index.isin(float_col_nan_indices[col]),col]
    model = linear_model.LinearRegression()
    model.fit(X, y)
    df.loc[df.index.isin(float_col_nan_indices[col]),col] = model.predict(df.loc[df.index.isin(float_col_nan_indices[col])].drop(columns=col))